Das CRISP DM Modell teilt sich in folgende Phasen auf:
<li> Business understanding
<li> Data understanding
<li> Data preparation
<li> Modeling
<li> Evaluating
<li> Deployment

In [3]:
# Benötigte Module importieren
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from xgboost import XGBClassifier

<h2> 1. Business understanding </h2>

Wir betrachten das Titanic-Problem. Basierend auf Passagierdaten wollen wir vorhersagen, ob jemand den Untergang der Titanic überlebt hätte. 




<h2> 2. Data understanding </h2>
Der Datensatz enthält Merkmale wie Geschlecht, Alter, Passagierklasse, Ticketpreis, und Anzahl der Familienmitglieder an Bord. 

In [4]:
# Daten einlesen
df = pd.read_csv('titanic.csv', delimiter=",")

In [5]:
# Welche Spalten sind im Datensatz vorhanden? 
# Wie viele Zeilen sind im Datensatz vorhanden?
# Wie viele fehlende Werte sind im Datensatz vorhanden?
# Welche Datentypen sind im Datensatz vorhanden?
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
# Statistische Kennzahlen anzeigen
df.describe().T

,count,mean,std,min,25%,50%,75%,max
PassengerId,891.0,446.000000,257.353842,1.00,223.5000,446.0000,668.5,891.0000
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292


In [7]:
# Menge der fehlenden Werte in den Spalten
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

<h2>3. Data preparation </h2>
Die Daten werden zur Modellierung vorbereitet

Aufgrund dessen, dass bei 891 Datenpunkten 687 mal die Datenpunkte zur Kabine fehlen, werden das Feature "Cabin" zur weiteren Analyse negiert <br>
Dies Spalte wird demnach zunächst aus dem Datensatz entfernt. Ebenfalls werden Spalten Ticket, PassengerId und Name entfernt, da diese als nicht relevant angesehen werden <br>
Anschließend wird der Datensatz in ein Ziel- und Featuredatensatz aufgeteilt

In [8]:
# Der Datensatz wird aufgeteilt in die Zielvariable und die Features
df_target = df['Survived']
df = df.drop(columns = ["Name", "Survived", "PassengerId", "Cabin", "Ticket"], axis = 1)

In [9]:
# Wir ersetzen die fehlenden Werte in der Spalte "Age" durch den Medianwert der Spalte
median_value = df['Age'].median() 
df['Age'] = df['Age'].fillna(median_value)

# Wir ersetzen die fehlenden Werte in der Spalte "Embarked" durch den am häufigsten vorkommenden Wert
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

In [10]:
# Die kategorischen Variablen werden in numerische Variablen umgewandelt
encoder = LabelEncoder()
categories = ['Sex', 'Embarked']

for i in categories:
    df[i] = encoder.fit_transform(df[i])

In [11]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# Es wird eine PCA durchgeführt, die Anzahl an relevanten Features weiter zu verringenn
pca = PCA(n_components=0.95)  # Keep 95% of variance
df_pca = pca.fit_transform(df_scaled)

print(f"Anteil der erklärten Varianz: {pca.explained_variance_ratio_}")
print(f"Anzahl Components: {pca.n_components_}")

pca_components = pd.DataFrame(
    pca.components_,
    columns=df.columns,  # Original feature names
    index=[f'PC{i+1}' for i in range(pca.n_components_)]  # PC names
)

print(pca_components)

Anteil der erklärten Varianz: [0.26512593 0.24494388 0.14233986 0.12044949 0.09520221 0.07958354
 0.05235509]
Anzahl Components: 7
       Pclass       Sex       Age     SibSp     Parch      Fare  Embarked
PC1 -0.533524 -0.343565  0.144889  0.188123  0.291205  0.614392 -0.280455
PC2  0.343899 -0.221583 -0.499894  0.541222  0.514095 -0.047083  0.152671
PC3 -0.200386  0.311234  0.408524  0.240118  0.192209  0.090660  0.770238
PC4 -0.003976  0.817780 -0.179913  0.327167 -0.034936  0.248950 -0.358632
PC5  0.286730  0.012434  0.685997  0.190487  0.371309 -0.326345 -0.407901
PC6  0.029428  0.247588 -0.138630 -0.678285  0.670581  0.093072  0.014296
PC7  0.688357 -0.076023  0.200107 -0.103142 -0.158717  0.659505  0.097803


Erkenntnisse der Dimensionsreduktion mit PCA:
<li>Pclass: Hat starke Beiträge zu PC1 und PC2, was sie zu einer wichtigen Variable macht, die beibehalten werden sollte.
<li>Sex: Trägt stark zu PC4 bei und hat auch Einfluss auf die Varianz, also sollte diese Variable ebenfalls beibehalten werden.
<li>Fare: Wichtiger Einfluss auf PC1 und sollte beibehalten werden.
<li>Age: Zeigt einen moderaten Einfluss auf mehrere PCs und könnte daher auch wichtig sein.
<li>Parch: Diese Variable hat in mehreren PCs (besonders in den späteren) geringe Ladewerte und scheint daher einen geringen Einfluss auf die Hauptkomponenten zu haben.
<li>SibSp: Auch diese Variable hat gemischte Ladewerte, aber insgesamt scheint sie weniger Einfluss auf die Varianz zu haben, die von den ersten PCs erklärt wird.
<li>Embarked: Diese Variable zeigt in PC3 eine starke positive Ladung (0.770), aber in den anderen PCs schwächere Einflüsse und kann daher eher vernachlässigt werden <br>
<br><b>Schlussfolgerung:</b> Die Merkmale Parch, Sibsp & Embarked werden ebenfalls aus dem Datensatz entfernt

In [12]:
#Spalten "SibSp", "Parch", "Embarked" wird entfernt
df = df.drop(columns=["SibSp", "Parch", "Embarked"], axis = 1)

<h2> 4. Modeling & Evaluierung - XGBoost </h2>
Es wird der K-nearest-neighbours Algorithmus angwendet, um die Daten zu modellieren. <br>
Für jeden Durchlauf des Modells wird eine Evaluierung durchgeführt.

In [13]:
states = [27, 6728, 49122]
features = df
target = df_target

In [27]:
accs = []

for RANDOM_STATE in states:
    # Daten aufteilen
    X_train, X_test, y_train, y_test = train_test_split(
        features, target, test_size=0.3, random_state=RANDOM_STATE
    )
        
    # Modell erstellen
    bst = XGBClassifier(n_estimators=5, max_depth=3)
    
    # Cross-Validation durchführen
    cv_scores = cross_val_score(bst, X_train, y_train, cv=10)
    mean_cv_score = np.mean(cv_scores)
    
    # Modell trainieren und testen
    bst.fit(X_train, y_train)
    preds = bst.predict(X_test)
    test_accuracy = accuracy_score(y_test, preds)
    accs.append(test_accuracy)
    
    # Ergebnisse ausgeben
    print(f"Random State: {RANDOM_STATE}")
    print(f"Cross-Validation Accuracy (Train): {mean_cv_score:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(classification_report(y_test, preds, zero_division=0.0))

# Durchschnittliche Genauigkeit und Standardabweichung
accs_mean = round(np.mean(accs), 4)
accs_std = round(np.std(accs), 4)

print(f"Mean Test Accuracy: {accs_mean}")
print(f"Std Test Accuracy: {accs_std}")


Random State: 27
Cross-Validation Accuracy (Train): 0.8329
Test Accuracy: 0.8134
              precision    recall  f1-score   support

           0       0.82      0.90      0.86       167
           1       0.80      0.67      0.73       101

    accuracy                           0.81       268
   macro avg       0.81      0.79      0.79       268
weighted avg       0.81      0.81      0.81       268

Random State: 6728
Cross-Validation Accuracy (Train): 0.8009
Test Accuracy: 0.8246
              precision    recall  f1-score   support

           0       0.82      0.92      0.87       168
           1       0.84      0.66      0.74       100

    accuracy                           0.82       268
   macro avg       0.83      0.79      0.80       268
weighted avg       0.83      0.82      0.82       268

Random State: 49122
Cross-Validation Accuracy (Train): 0.7865
Test Accuracy: 0.8358
              precision    recall  f1-score   support

           0       0.85      0.91      0.87

<h2> 5. Deployment </h2>

<li>Zunächst wurden die Daten aufbereitet, um ein Verständnis für sie zu entwickeln. Es lagen Daten im numerischen und kategorischen Format vor
<li>Wir haben verschiedene Datensätze ausprobiert und getestet, um die höchste Genauigkeit zu erzielen, und sind zu dem Schluss gekommen, dass die Merkmale Pclass, Sex, Age und Fare die besten Ergebnisse liefern
<li>Die anderen Merkmale wurden aus dem Modell entfernt
<li>Das Modell wurde mit drei verschiedenen Random States durchgeführt, um die Robustheit der Ergebnisse zu prüfen.</li> 
<li>Da die Testgenauigkeit bei allen Random States sehr ähnlich ist (81,34% bis 83,58%) und die Standardabweichung der Genauigkeit gering ist (±0,91%), kann das Modell als robust und nicht anfällig für Overfitting angesehen werden.</li> 
<li>Die Ergebnisse zeigen eine durchschnittliche Testgenauigkeit von **82,46%**, wobei die höchste Genauigkeit bei Random State 49122 mit **83,58%** erreicht wurde.</li> <li>Die Cross-Validation-Ergebnisse stützen diese Beobachtungen, da die durchschnittlichen Validierungsgenauigkeiten zwischen **78,65%** und **83,29%** liegen. Dies zeigt, dass das Modell auch bei unterschiedlichen Aufteilungen der Trainingsdaten verlässlich bleibt.</li> 
<li>Das Modell zeigt eine starke Leistung bei der Erkennung von Nicht-Überlebenden (Klasse 0) mit einem Recall von 90% und einer Präzision von bis zu 85%. Die Vorhersage von Überlebenden (Klasse 1) ist jedoch schwächer, mit einem Recall von 66% bis 71% und einer Präzision von 80% bis 84%.</li> 
<li>Insgesamt liefert das Modell robuste Vorhersagen, insbesondere für Nicht-Überlebende, mit Potenzial zur Optimierung der Vorhersagen für Überlebende.</li>